# Changing the Insta algorithm actively

This script when provided with the user account and password, it will login and based on the intrests provided, it will find users to follow. This allows us to have a better algorithm that is able to be aline with our goals

**Not Recommned to use as Instagram detects that this isnt a human logging in!**

In [1]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.14.0
    Uninstalling typing_extensions-4.14.0:
      Successfully uninstalled typing_extensions-4.14.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 r

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Function to log in to Instagram
def login(username, password):
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

    driver = webdriver.Chrome(options=options)  # Make sure to have the ChromeDriver installed
    driver.get("https://www.instagram.com/accounts/login/")

    # Wait for the login fields to be present
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, "username")))

    # Enter username and password
    driver.find_element(By.NAME, "username").send_keys(username)
    driver.find_element(By.NAME, "password").send_keys(password)
    driver.find_element(By.XPATH, "//button[@type='submit']").click()

    # Wait for login to complete
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//nav")))
    return driver

# Function to follow users
def follow_users(driver, user_list):
    for user in user_list:
        driver.get(f"https://www.instagram.com/{user}/")

        # Wait for the Follow button to be clickable
        try:
            follow_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//button[text()='Follow']"))
            )
            follow_button.click()
            print(f"Followed {user}")
        except Exception as e:
            print(f"Could not follow {user}: {e}")

        # Random delay between actions
        time.sleep(random.randint(60, 120))  # Random delay between 1 to 2 minutes

# Function to get user suggestions from a conversational LLM
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

def get_user_suggestions_from_llm(topics):
    # Load the model and tokenizer
    model_name = "microsoft/Phi-4-mini-instruct"  # Use your desired model
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Check if GPU is available and set the device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Clear GPU cache
    torch.cuda.empty_cache()

    # Load the model with low CPU memory usage and in half precision if possible
    model = AutoModelForCausalLM.from_pretrained(model_name, low_cpu_mem_usage=True, torch_dtype=torch.float16).to(device)

    suggested_users = []
    for topic in topics:
        # Prepare the input for the model
        input_text = "Based on the following topics, suggest Instagram accounts to follow on Instagram: " + ", ".join(topic)
        input_ids = tokenizer.encode(input_text + tokenizer.eos_token, return_tensors='pt').to(device)  # Move input to GPU

        # Generate a response
        chat_history_ids = model.generate(input_ids, max_length=100, pad_token_id=tokenizer.eos_token_id)
        response = tokenizer.decode(chat_history_ids[:, input_ids.shape[-1]:][0], skip_special_tokens=True)

        # Parse the response to extract usernames (assuming they are mentioned in the response)
        suggested_users.append([user.strip() for user in response.split(",") if user.strip()])

        # Clear unused variables and cache
        del input_ids, chat_history_ids, response
        torch.cuda.empty_cache()

    return suggested_users



# Main execution
if __name__ == "__main__":
    username = "your_username"  # Replace with your username
    password = "your_password"    # Replace with your password

    # Get topics from the user
    topics_input = input("Enter topics of interest (comma-separated): ")
    print(topics_input)
    topics = [topic.strip() for topic in topics_input.split(",")]

    # Get user suggestions from the LLM
    users_to_follow = get_user_suggestions_from_llm(topics)
    print(f"Users to follow based on your interests: {users_to_follow}")

    # Log in and follow users
    #driver = login(username, password)
    #follow_users(driver, users_to_follow)
    #driver.quit()


Enter topics of interest (comma-separated): aws,docker,basketball,linux
aws,docker,basketball,linux


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Users to follow based on your interests: [['# The Art of Writing a Good Summary\n\nWriting a good summary is an essential skill for students', 'professionals', 'and anyone who needs to condense information into a concise and coherent form. A well-crafted summary captures the main ideas and key details of a text while eliminating unnecessary information. In this article', 'we will explore the art of writing a good summary', 'providing tips and techniques to help you master this'], ['# The Art of Writing a Good Summary\n\nWriting a good summary is an essential skill for students', 'professionals', 'and anyone who needs to condense information into a concise and coherent form. A well-crafted summary captures the essence of the original text while maintaining its meaning and intent. In this article', 'we will explore the art of writing a good summary', 'focusing on the key'], ['# The Art of Writing Unit Tests in Python\n\nUnit testing is a crucial part of software development. It helps ens